# Programming constructs as data structures

- Data literals: e.g. 3.1415, "Hello world".
- Arithmetic expressions: `+`, `-`, `*`, `/`.
- Variables:
    - Referencing variables: `pi`
    - Assignment of variables: `pi = 3.1415`
- Logical conditions: `pi < 3`
- If/Else branching

# Everything is an instance of `Expr` class.

In [1]:
#
# Expr base class
#
class Expr:
    def evaluate(self, context):
        raise Exception("Not implement")

In [2]:
#
# Literal
#

class Literal(Expr):
    def __init__(self, value):
        self.value = value
        
    def evaluate(self, context):
        return self.value

In [5]:
###
# 3.1415
###

e = Literal(3.1415)
e.evaluate(None)

3.1415

In [6]:
#
# Arith - mathematical operations over two expressions
#

class Arith(Expr):
    math_ops = {
        'plus': lambda x,y:x + y,
        'sub': lambda x, y: x - y,
        'mult': lambda x, y: x * y,
        'div': lambda x, y: x / y
    }
    
    def __init__(self, left:Expr, op:str, right:Expr):
        self.left = left
        self.op = op
        self.right = right
    
    def evaluate(self, context):
        if self.op in self.math_ops:
            f = self.math_ops[self.op]
            x = self.left.evaluate(context)
            y = self.right.evaluate(context)
            return f(x, y)
        else:
            raise Exception("Unknown operator: %s" % self.op)

In [8]:
####
# Area of a circle with radius of 10.2
# 3.1415 * (10.2 * 10.2)
####

Arith(
    Literal(3.1415), 
    'mult',
    Arith(
        Literal(10.2),
        'mult',
        Literal(10.2))).evaluate(None)

326.84166

# Definition of a context

> A context is a mapping of variable (names) to their data values.

This is also known as a _scope_ (refer to _Programming Languages_).

In [9]:
#
# Manually create a context
#
C = {
    "pi": 3.1415,
    "radius": 10.2
}

In [10]:
#
# Variable
#

class Var(Expr):
    def __init__(self, name):
        self.name = name
        
    def evaluate(self, context):
        if self.name in context:
            return context[self.name]
        else:
            raise Exception("Variable is not defined in context: %s" % self.name)

In [13]:
#####
# Looking up the value of pi by variable reference:
# pi
#####
Var("pi").evaluate(C)

3.1415

In [16]:
####
# Calculate the area of the circle using variables
# pi * radius * radius
####

Arith(Var("pi"), 'mult', Arith(Var("radius"), 'mult', Var("radius"))).evaluate(C)

326.84166

In [17]:
#
# Assignment to variables
#

class Assign(Expr):
    def __init__(self, name:str, expr:Expr):
        self.name = name
        self.expr = expr
        
    def evaluate(self, context):
        value = self.expr.evaluate(context)
        context[self.name] = value

In [19]:
#####
# i = 0
#####
Assign("i", Literal(0)).evaluate(C)

In [20]:
C

{'pi': 3.1415, 'radius': 10.2, 'i': 0}

In [22]:
#####
# i = i + 1
#####
Assign("i", Arith(Var("i"), 'plus', Literal(1))).evaluate(C)

In [23]:
C

{'pi': 3.1415, 'radius': 10.2, 'i': 1}

# Branching

- Logical condition expressions
- IfElse expressions

In [25]:
#
# Conditions
#

logical_cond = {
    'lt': lambda x,y: x < y,
    'gt': lambda x,y: x > y,
    'ne': lambda x,y: not(x == y),
    'eq': lambda x,y: x == y,
    'and': lambda x,y: x and y,
    'or': lambda x,y: x or y
}

class Cond(Arith):
    def evaluate(self, context):
        x = self.left.evaluate(context)
        y = self.right.evaluate(context)
        return logical_cond[self.op](x, y)

In [31]:
######
# radius > 10
######
print("C = ", C)
Cond(Var("radius"), 'gt', Literal(10)).evaluate(C)

C =  {'pi': 3.1415, 'radius': 10.2, 'i': 1}


True

In [32]:
#
# Branching expression with if-else
#

class IfElse(Expr):
    def __init__(self, cond:Cond, if_expr:Expr, else_expr:Expr):
        self.cond = cond
        self.if_expr = if_expr
        self.else_expr = else_expr
    
    def evaluate(self, context):
        if self.cond.evaluate(context):
            return self.if_expr.evaluate(context)
        else:
            return self.else_expr.evalaute(context)

In [35]:
#######
# if radius > 10 then
#   "Greater than 10"
# else
#   "not bigger than 10"
#######

e = IfElse(
    Cond(Var("radius"), 'gt', Literal(10)),
    Literal("Greater than 10"),
    Literal("not bigger than 10")
)

print("C = ", C)
e.evaluate(C)

C =  {'pi': 3.1415, 'radius': 10.2, 'i': 1}


'Greater than 10'

# TODO

- Looping with while-loop (for loop, do-while loops)
- User defined functions
- Support types